In [ ]:
# onnx models with double precision

# 1) try to run a Conv operator with double precision
#    RESULT: not possible for conv
# 
# 2) test precision with custom made double matrix
#    RESULT: <TODO>
#



In [1]:
import io
import numpy as np
import onnx
import onnxruntime
import matplotlib.pyplot as plt
import tenseal as ts
import torch

from onnx import helper
from onnx import numpy_helper
from onnx import TensorProto
from torch import nn
from torch.onnx import export as export_onnx

In [2]:
input_length = 28
OPSET_VERSION = 14

In [5]:
# 2D Conv
n_inputs = input_length * input_length
n_channels_in = 1
n_channels_out = 4
kernel_size = 7
stride = 3
model = nn.Conv2d(n_channels_in, n_channels_out, kernel_size, stride, dtype=torch.double)

export_onnx(
    model,
    torch.empty([1, 1, input_length, input_length], dtype=torch.double),
    "./graphs/conv_double.onnx",
    opset_version=OPSET_VERSION,
)

In [6]:
model_path = "./graphs/conv_double.onnx"
# model_path = "../tenseal-inference/tests/models/conv.onnx"
# model_path = "../tenseal-inference/models/mnist.onnx"

model = onnx.load(model_path)

In [9]:
np.random.seed(27)
x = np.asarray(np.random.random([1, n_channels_in, input_length, input_length]), np.double)

w = numpy_helper.to_array(model.graph.initializer[0])
b = numpy_helper.to_array(model.graph.initializer[1])

result_dict = {}

In [17]:
atts = {}
for a in model.graph.node[0].attribute:
    if a.name == "group":
        atts[a.name] = a.i
    else:
        atts[a.name] = a.ints
        
n_dims = len(x.shape[2:])
out_shape = [
    int(
        (input_length + sum(atts["pads"][i::n_dims]) - atts["dilations"][i] * (atts["kernel_shape"][i] - 1) - 1)
        / atts["strides"][i] + 1
    )
    for i in range(n_dims)
]

# out_shape_x = (input_length - atts["kernel_shape"][0] + atts["pads"][0] + atts["pads"][2]) // atts["strides"][0] + 1
# out_shape_y = (input_length - atts["kernel_shape"][1] + atts["pads"][1] + atts["pads"][3]) // atts["strides"][1] + 1        

X = helper.make_tensor_value_info("X", TensorProto.DOUBLE, (None, n_channels_in, input_length, input_length))
Y = helper.make_tensor_value_info("Y", TensorProto.DOUBLE, (None, n_channels_out) + tuple(out_shape))

# Create Conv node
node_def = helper.make_node(
    "Conv",  # node name
    ["X", "W"],  # inputs (B is optional and dropped as it has to be 0)
    ["Y"],  # outputs
    dilations=atts["dilations"],
    group=atts["group"],
    kernel_shape=atts["kernel_shape"],
    pads=atts["pads"],
    strides=atts["strides"],
)

weight_init = onnx.helper.make_tensor(
    name="W",
    data_type=onnx.mapping.NP_TYPE_TO_TENSOR_TYPE[w.dtype],
    dims=w.shape,
    vals=w.flatten(),
)

# Create the graph (GraphProto)
graph_def = helper.make_graph([node_def], "conv-model", [X], [Y], initializer=[weight_init])

model_def = helper.make_model(graph_def)
model_def.opset_import[0].version = OPSET_VERSION
onnx.checker.check_model(model_def)
buffer = io.BytesIO()
onnx.save(model_def, buffer)
session = onnxruntime.InferenceSession(buffer.getvalue())

# run convolution with identity matrix as input to get convolution matrix
eye = np.eye(input_length * input_length, dtype=w.dtype).reshape(input_length * input_length, n_channels_in, input_length, input_length)
M_conv = np.array(session.run(None, {"X": eye})).reshape(
    input_length * input_length, n_channels_out * np.prod(out_shape)
)

bias_conv = np.repeat(b, np.prod(out_shape))

x3 = x.reshape(1,-1)
y3 = x3 @ M_conv + bias_conv

result_dict["numpy"] = y3.ravel()

NotImplemented: [ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Conv(11) node with name ''